# はじめに
機械学習を用いる場合、様々なライブラリやフレームワークを使いますが、日本語の文書を対象にする場合は少し注意が必要なことがあります。

今回、日本語の文書を対象に、scikit-learn (0.24.2)のCountVectorizerを用いてベクトル化する時の、analyzerオプションとtokenizerオプションの違いについてまとめます。

## 英語の文書の場合
CountVectorizerは、単語が空白文字で区切られる文書を想定しています。そのため、日本語の文書をデフォルトの設定で分かち書きしようとすると、うまくいきません。

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

# コーパス
# 文は Japan Times より。
all_sents=['The top court on Wednesday again ruled that legal provisions forcing married couples to use the same surname are constitutional, upholding a Supreme Court judgment from 2015.',
            'Japan is suspending approval for companies to inoculate staff amid concerns that an increase in such applications will hamper the smooth delivery of shots, Taro Kono, the minister in charge of vaccines, said Wednesday.']


In [26]:
# まずは英語の単語(1-gram) 
# analyzer に'word'を指定(デフォルト)

cv = CountVectorizer()
counts = cv.fit_transform(all_sents)
print(cv.get_feature_names())

['2015', 'again', 'amid', 'an', 'applications', 'approval', 'are', 'charge', 'companies', 'concerns', 'constitutional', 'couples', 'court', 'delivery', 'for', 'forcing', 'from', 'hamper', 'in', 'increase', 'inoculate', 'is', 'japan', 'judgment', 'kono', 'legal', 'married', 'minister', 'of', 'on', 'provisions', 'ruled', 'said', 'same', 'shots', 'smooth', 'staff', 'such', 'supreme', 'surname', 'suspending', 'taro', 'that', 'the', 'to', 'top', 'upholding', 'use', 'vaccines', 'wednesday', 'will']


In [30]:
# 英語の 2-gram 
cv = CountVectorizer(ngram_range=(2, 2)) # (1, 2)とすると 1-gram と 2-gram の和集合 
counts = cv.fit_transform(all_sents)
print(cv.get_feature_names())

['again ruled', 'amid concerns', 'an increase', 'applications will', 'approval for', 'are constitutional', 'charge of', 'companies to', 'concerns that', 'constitutional upholding', 'couples to', 'court judgment', 'court on', 'delivery of', 'for companies', 'forcing married', 'from 2015', 'hamper the', 'in charge', 'in such', 'increase in', 'inoculate staff', 'is suspending', 'japan is', 'judgment from', 'kono the', 'legal provisions', 'married couples', 'minister in', 'of shots', 'of vaccines', 'on wednesday', 'provisions forcing', 'ruled that', 'said wednesday', 'same surname', 'shots taro', 'smooth delivery', 'staff amid', 'such applications', 'supreme court', 'surname are', 'suspending approval', 'taro kono', 'that an', 'that legal', 'the minister', 'the same', 'the smooth', 'the top', 'to inoculate', 'to use', 'top court', 'upholding supreme', 'use the', 'vaccines said', 'wednesday again', 'will hamper']


## 日本語の文書の場合
日本語の文書をデフォルトの設定で分かち書きしようとすると、うまくいきません。tokenizerオプションに分かち書きをする関数を渡しますが、analyzerに渡してもうまくいきます。ネットで探すとanalyzerに渡した例のほうが多いように思います。

1-gramの場合はどちらのオプションにしても問題ありませんが、ngram_rangeを使ってn-gramを使いたい場合は、分かち書きの関数はtokenizerに渡す必要があります。

これは、[CountVectorizerのマニュアル](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)のngram_rangeのところに、以下のように書いてある通り、ngram_rangeを有効にするにはanalyzerには関数は渡せないからです。
> Only applies if analyzer is not callable.

In [31]:
# 日本語のコーパス
# 文は読売オンラインより
all_sents = ['東京五輪・パラリンピック大会組織委員会は２３日、販売済みのチケットの一部について再抽選を実施すると発表した。',
            '　河野行政・規制改革相がテレビ番組に連日出演し、若者に新型コロナウイルスワクチンの接種を呼びかけている。']

In [32]:
# 分かち書きをする関数を定義
import MeCab

#tagger = MeCab.Tagger('-d /opt/local/lib/mecab/dic/ipadic-utf8 -Owakati')
tagger = MeCab.Tagger('-Owakati')
tagger.parse('')

def sent2words(sent):
  node = tagger.parseToNode(sent)

  word_list = []
  while node:
    meta = node.feature.split(',')
    if meta[0] == '名詞':
        word_list.append(node.surface)
    node = node.next

  return(word_list)

# テスト
for i in all_sents:
    print(sent2words(i))

['東京', '五輪', 'パラリンピック', '大会', '組織', '委員', '会', '２', '３', '日', '販売', '済み', 'チケット', '一部', '抽選', '実施', '発表']
['河野', '行政', '規制', '改革', '相', 'テレビ', '番組', '連日', '出演', '若者', '新型', 'コロナウイルスワクチン', '接種']


In [33]:
# デフォルトの設定ではうまく単語に分割できない。
cv = CountVectorizer()
counts = cv.fit_transform(all_sents)
print(cv.get_feature_names())

['パラリンピック大会組織委員会は２３日', '東京五輪', '河野行政', '若者に新型コロナウイルスワクチンの接種を呼びかけている', '規制改革相がテレビ番組に連日出演し', '販売済みのチケットの一部について再抽選を実施すると発表した']


### analyzerに関数を渡す
日本語文書で1-gramを使う場合、analyzerに分かち書きの関数を渡せばよい。ただし、以下で見るように、n-gramを使う場合、このやり方ははうまくいかない。

In [34]:
# analyzer に分かち書きの関数を指定
cv = CountVectorizer(analyzer=sent2words)
counts = cv.fit_transform(all_sents)
print(cv.get_feature_names())

['コロナウイルスワクチン', 'チケット', 'テレビ', 'パラリンピック', '一部', '五輪', '会', '出演', '大会', '委員', '実施', '抽選', '接種', '改革', '新型', '日', '東京', '河野', '済み', '番組', '発表', '相', '組織', '若者', '行政', '規制', '販売', '連日', '２', '３']


In [35]:
# analyzer に関数を指定した上で ngram_range を指定
cv = CountVectorizer(analyzer=sent2words, ngram_range=(1, 2))
counts = cv.fit_transform(all_sents)
print(cv.get_feature_names())

['コロナウイルスワクチン', 'チケット', 'テレビ', 'パラリンピック', '一部', '五輪', '会', '出演', '大会', '委員', '実施', '抽選', '接種', '改革', '新型', '日', '東京', '河野', '済み', '番組', '発表', '相', '組織', '若者', '行政', '規制', '販売', '連日', '２', '３']


### tokenizerに関数を渡す
gram_rangeを使う場合は、分かち書きの関数はanalyzerではなくtokenizerに渡す必要がある。


In [36]:
# tokenizer に分かち書きの関数を指定
cv = CountVectorizer(tokenizer=sent2words)
counts = cv.fit_transform(all_sents)
print(cv.get_feature_names())

['コロナウイルスワクチン', 'チケット', 'テレビ', 'パラリンピック', '一部', '五輪', '会', '出演', '大会', '委員', '実施', '抽選', '接種', '改革', '新型', '日', '東京', '河野', '済み', '番組', '発表', '相', '組織', '若者', '行政', '規制', '販売', '連日', '２', '３']


In [38]:
# tokenizer に関数を指定した上で ngram_range を指定
cv = CountVectorizer(tokenizer=sent2words, ngram_range=(1, 2))
counts = cv.fit_transform(all_sents)
print(cv.get_feature_names())

['コロナウイルスワクチン', 'コロナウイルスワクチン 接種', 'チケット', 'チケット 一部', 'テレビ', 'テレビ 番組', 'パラリンピック', 'パラリンピック 大会', '一部', '一部 抽選', '五輪', '五輪 パラリンピック', '会', '会 ２', '出演', '出演 若者', '大会', '大会 組織', '委員', '委員 会', '実施', '実施 発表', '抽選', '抽選 実施', '接種', '改革', '改革 相', '新型', '新型 コロナウイルスワクチン', '日', '日 販売', '東京', '東京 五輪', '河野', '河野 行政', '済み', '済み チケット', '番組', '番組 連日', '発表', '相', '相 テレビ', '組織', '組織 委員', '若者', '若者 新型', '行政', '行政 規制', '規制', '規制 改革', '販売', '販売 済み', '連日', '連日 出演', '２', '２ ３', '３', '３ 日']


### まとめ
最後のセルでは、(1, 2)-gramに分けたので、1つの単語のみと、二つの単語が空白で連結されたものが
出力されることが分かります。